In [1]:
# import needed toolboxes
from HatLabNWBConverters import MarmForageNWBConverter
from pprint import pprint
from pathlib import Path
from nwb_conversion_tools.utils.metadata import load_metadata_from_file
from nwb_conversion_tools.utils.json_schema import dict_deep_update
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget

In [13]:
# User defined inputs
dpath = '/Users/paul/Documents/Data/'
sess = 'TY20210403_0656_inHammock_morning/'
rfile = 'TY20210403_0656_inHammock_morning-001.ns6'
nfile = 'TY20210403_0656_inHammock_morning-001.nev'
mfile = 'metadata.yml'
sfile = 'tony.yml'
ofile = 'testcheck3.nwb'

In [3]:
# Get source_schema
source_schema = MarmForageNWBConverter.get_source_schema()
pprint(source_schema, width=120)

{'$id': 'source.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': False,
 'description': 'Schema for the source data, files and directories',
 'properties': {'BlackrockRecordingInterfaceRaw': {'additionalProperties': False,
                                                   'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                               'format': 'file',
                                                                               'type': 'string'}},
                                                   'required': ['filename'],
                                                   'type': 'object'},
                'BlackrockSortingInterface': {'additionalProperties': True,
                                              'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                          'f

In [4]:
# Construct the source data
base_path = Path(dpath+sess)
movie_path = Path(dpath+sess+'videos/')
file_recording_raw = str(base_path / rfile)
file_sorting = str(base_path / nfile)
kmovie_files = list(movie_path.glob('*.avi'))

source_data = dict(
    BlackrockRecordingInterfaceRaw=dict(filename=file_recording_raw),
    BlackrockSortingInterface=dict(
        filename=file_sorting,
        nsx_to_load=6
    )
)

movies = list()
if len(kmovie_files)>0:
    for movie in kmovie_files:
        movies.append(str(movie))
    source_data.update(KinematicVideos=dict(file_paths=movies))    

pprint(source_data, width=120)

{'BlackrockRecordingInterfaceRaw': {'filename': '/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/TY20210403_0656_inHammock_morning-001.ns6'},
 'BlackrockSortingInterface': {'filename': '/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/TY20210403_0656_inHammock_morning-001.nev',
                               'nsx_to_load': 6},
 'KinematicVideos': {'file_paths': ['/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/videos/TYJL_2021_03_07_preCrickets_session1_event003_cam1.avi',
                                    '/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/videos/TYJL_2021_03_07_preCrickets_session1_event002_cam1.avi',
                                    '/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/videos/TYJL_2021_03_07_preCrickets_session1_event002_cam2.avi',
                                    '/Users/paul/Documents/Data/TY20210403_0656_inHammock_morning/videos/TYJL_2021_03_07_preCrickets_session1_event003_cam2.avi'

In [5]:
# Initialize converter
converter = MarmForageNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint(converter.data_interface_objects, width=120)

Source data is valid!
Data interfaces for this converter:
{'BlackrockRecordingInterfaceRaw': <nwb_conversion_tools.datainterfaces.ecephys.blackrock.blackrockdatainterface.BlackrockRecordingExtractorInterface object at 0x7fd204d8bb80>,
 'BlackrockSortingInterface': <nwb_conversion_tools.datainterfaces.ecephys.blackrock.blackrockdatainterface.BlackrockSortingExtractorInterface object at 0x7fd202e9ff10>,
 'KinematicVideos': <nwb_conversion_tools.datainterfaces.behavior.movie.moviedatainterface.MovieInterface object at 0x7fd204d8bd30>}


In [6]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
#pprint(metadata_schema, width=120)

In [7]:
# Get metadata from source data and from file
metadata = converter.get_metadata()
# load exp metadata
m = load_metadata_from_file(dpath+sess+'/'+mfile)
metadata = dict_deep_update(metadata, m)
# load subject metadata
s = load_metadata_from_file(dpath+sess+'/'+sfile)
metadata = dict_deep_update(metadata, s)
# convert 
converter.validate_metadata(metadata)
pprint(metadata, width=120)

Metadata is valid!
{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw'},
             'ElectrodeGroup': [{'description': 'no description',
                                 'device': 'Device_ecephys',
                                 'location': 'unknown',
                                 'name': '0'}]},
 'NWBFile': {'experimenter': ['Jeff Walker'],
             'identifier': 'aeee9b59-4780-47b0-8b4d-c7131bf67c09',
             'institution': 'University of Chicago',
             'lab': 'Hatsopoulos Lab',
             'session_description': '5 minute early morning recording check',
             'session_start_time': '2021-04-03T06:56:00-05:00',
             'surgery': 'No surgery for this recording'},
 'Subject': {'date_of_birth': '2018-01-01T00:00:00-05:00',
             'sex': 'male',
             'species': 'marmoset',
             'subject_id': 'Tony',
             'weight': 350.0}

In [8]:
# conversion schema
conversion_options_schema = converter.get_conversion_options_schema()
print("Conversion options for each data interface: \n")
pprint(conversion_options_schema['properties'], width=120)

Conversion options for each data interface: 

{'BlackrockRecordingInterfaceRaw': {'additionalProperties': False,
                                    'properties': {'buffer_mb': {'default': 500, 'type': 'number'},
                                                   'es_key': {'type': 'string'},
                                                   'overwrite': {'default': False, 'type': 'boolean'},
                                                   'save_path': {'type': 'string'},
                                                   'stub_test': {'default': False, 'type': 'boolean'},
                                                   'use_times': {'default': False, 'type': 'boolean'},
                                                   'write_as': {'default': 'raw', 'type': 'string'}},
                                    'required': [],
                                    'type': 'object'},
 'BlackrockSortingInterface': {'additionalProperties': False,
                               'properties

In [10]:
# build conversion options
conversion_options = converter.get_conversion_options()
conversion_options['BlackrockRecordingInterfaceRaw']=dict(es_key='raw recording')
conversion_options['BlackrockSortingInterface']=dict(write_ecephys_metadata=True)
# get video start times
#conversion_options['KinematicVideos'] = dict(starting_times=kstart_times)
pprint(conversion_options, width=120)

{'BlackrockRecordingInterfaceRaw': {'es_key': 'raw recording'},
 'BlackrockSortingInterface': {'write_ecephys_metadata': True},
 'KinematicVideos': {}}


In [11]:
pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw'},
             'ElectrodeGroup': [{'description': 'no description',
                                 'device': 'Device_ecephys',
                                 'location': 'unknown',
                                 'name': '0'}]},
 'NWBFile': {'experimenter': ['Jeff Walker'],
             'identifier': 'aeee9b59-4780-47b0-8b4d-c7131bf67c09',
             'institution': 'University of Chicago',
             'lab': 'Hatsopoulos Lab',
             'session_description': '5 minute early morning recording check',
             'session_start_time': '2021-04-03T06:56:00-05:00',
             'surgery': 'No surgery for this recording'},
 'Subject': {'date_of_birth': '2018-01-01T00:00:00-05:00',
             'sex': 'male',
             'species': 'marmoset',
             'subject_id': 'Tony',
             'weight': 350.0}}


In [14]:
converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=ofile, 
    save_to_file=True,
    conversion_options=None
)

Metadata is valid!
Converting Blackrock raw traces...
NWB file saved at testcheck3.nwb!


In [15]:
io = NWBHDF5IO(ofile, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)